In [87]:
import pandas as pd
import numpy as np
#import findspark
#findspark.init()
from pyspark.sql import SparkSession , SQLContext
from pyspark import SparkContext,SparkConf
import plotly.graph_objects as go
import plotly.express as px
from pyspark.ml.stat import Summarizer
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import StructType, StructField,StringType, IntegerType, BooleanType, BinaryType
from pyspark.mllib.linalg import SparseVector
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VarianceThresholdSelector
from pyspark.ml.linalg import Vectors

In [40]:
SRR_aMetagenomes=pd.read_table("Metadata/aMetagenomes/SRR_aMetagenomes.txt", names=["Accession"],)

In [41]:
aMetagenomes_SRStoSRR=pd.read_table("Metadata/aMetagenomes/aMetagenomes_SRStoSRR.tsv",names=["archive_accession","SRR_accession"])
aMetagenomes=aMetagenomes_SRStoSRR[aMetagenomes_SRStoSRR['SRR_accession'].notna()]

In [42]:
aMetagenomes_metadata=pd.read_table("Metadata/aMetagenomes/aMetagenomes.txt")
#Create one row per archive_accession
aMetagenomes_metadata=aMetagenomes_metadata.assign(archive_accession=aMetagenomes_metadata['archive_accession'].str.split(',')).explode('archive_accession')
aMetagenomes_metadata=aMetagenomes_metadata.reset_index()
aMetagenomes_metadata=aMetagenomes_metadata[aMetagenomes_metadata["archive"]=="SRA"]
aMetagenomes_metadata.shape

(447, 18)

In [43]:
aMetagenomes_metadata.head()

,index,project_name,publication_year,publication_doi,site_name,latitude,longitude,geo_loc_name,sample_name,sample_host,sample_age,sample_age_doi,community_type,material,collection_date,archive,archive_project,archive_accession
0,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473742
1,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473743
2,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473744
3,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473745
4,2,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,G12,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473747


In [44]:
len(aMetagenomes_SRStoSRR.SRR_accession)

518

In [45]:
aMetagenomes_metadata.shape

(447, 18)

In [46]:
#Filter aMetagenomes_metadata with only files that were downloaded

In [47]:
SRR_aMetagenomes=pd.read_table("Metadata/aMetagenomes/SRR_aMetagenomes.txt",names=["SRR_accession"])

In [48]:
a=pd.merge(aMetagenomes_SRStoSRR,SRR_aMetagenomes,on="SRR_accession",how="inner")


In [49]:
aMetagenomes_metadata=pd.merge(aMetagenomes_metadata,aMetagenomes_SRStoSRR,on="archive_accession",how="inner")

In [50]:
fig = px.histogram(aMetagenomes_metadata, x="sample_age",title="Sample age")
fig.show()

In [51]:
fig = px.histogram(aMetagenomes_metadata, x="sample_host",title="Sample host")
fig.show()

In [52]:
fig = px.histogram(aMetagenomes_metadata, x="community_type",title="Community type")
fig.show()

In [53]:
fig = px.histogram(aMetagenomes_metadata, x="material",title="material")
fig.show()

In [54]:
fig = px.scatter_mapbox(aMetagenomes_metadata, lat="latitude", lon="longitude", hover_name="geo_loc_name",
                        hover_data=['site_name'],
                        color_discrete_sequence=["red"], zoom=2, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# PySPARK

In [102]:
# Define session
#The following line should be run if the last session was not closed
sc.stop()
conf = SparkConf().setAll([("spark.executors", "5"),("spark.executors.memory","90g"),\
("spark.executor.cores","5"),("spark.driver.memory","8000g"),("spark.local.dir","/pasteur/sonic/scratch/public/cduitama/RascovanProject/tmp/")])
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [104]:
print("Spark version is "+ sc.version)

Spark version is 3.1.1


In [105]:
#What is the actual configuration of the session
sc.getConf().getAll()

[('spark.executors.memory', '90g'),
 ('spark.executor.cores', '5'),
 ('spark.driver.host', 'maestro-2499.maestro.pasteur.fr'),
 ('spark.executors', '5'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.local.dir',
  '/pasteur/sonic/scratch/public/cduitama/RascovanProject/tmp/'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8000g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.driver.port', '44285'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.startTime', '1614776631746'),
 ('spark.app.id', 'local-1614776631809'),
 ('spark.ui.showConsoleProgress', 'true')]

In [106]:
schema = StructType([ \
    StructField("Kmer",StringType(),True)])
for each in list(SRR_aMetagenomes.SRR_accession):
    schema.add(each,IntegerType(),True)

In [107]:
aMetagenomes=spark.read.csv("kmMatrices/aMetagenomes/whole_aMetagenomes_kmMatrix.txt",header=False, sep=" ",schema=schema)

In [108]:
aMetagenomes.createTempView("aMetagenomes")

In [109]:
aMetagenomes.columns[1:10]

['SRR957738',
 'SRR957739',
 'SRR957740',
 'SRR957741',
 'SRR957742',
 'SRR957743',
 'SRR957744',
 'SRR957745',
 'SRR957746']

In [110]:
colnames=["Kmer"]+list(SRR_aMetagenomes.SRR_accession)

In [111]:
aMetagenomes=aMetagenomes.toDF(*colnames)
aMetagenomes.columns[1:10]

['SRR957738',
 'SRR957739',
 'SRR957740',
 'SRR957741',
 'SRR957742',
 'SRR957743',
 'SRR957744',
 'SRR957745',
 'SRR957746']

In [112]:
aMetagenomes.limit(5).toPandas()


,Kmer,SRR957738,SRR957739,SRR957740,SRR957741,SRR957742,SRR957743,SRR957744,SRR957745,SRR957746,...,SRR5047099,SRR5047084,SRR5047171,SRR5047158,SRR5047135,SRR5047202,SRR5047105,SRR5047103,SRR5047142,SRR5047064
0,AAAAAAAAAAAAGAGCACAC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AAAAAAAAAAAGAGCACACA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAAAAAAAAAAGAGCACACC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AAAAAAAAAAAGAGCACACT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AAAAAAAAAAAGAGCACACG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
aMetagenomes.limit(5).toPandas()["Kmer"][1]

'AAAAAAAAAAAGAGCACACA'

In [114]:
#Here I am selecting only the Oral samples from Homo sapiens

In [115]:
HS_Oral=aMetagenomes_metadata[(aMetagenomes_metadata["sample_host"]=="Homo sapiens")\
                                  & (aMetagenomes_metadata["community_type"]=="oral")]

In [116]:
SRR_HS_Oral=["Kmer"]+list(HS_Oral["SRR_accession"])

In [117]:
HS_Oral_kmMatrix=aMetagenomes.select(*SRR_HS_Oral).cache()

In [118]:
HS_Oral_kmMatrix.createTempView("HS_Oral_kmMatrix")

In [119]:
#This is how the matrix of oral human samples looks like, we are left with 282 samples
HS_Oral_kmMatrix.limit(5).toPandas()

,Kmer,SRR957738,SRR957739,SRR957740,SRR957741,SRR957742,SRR957743,SRR957744,SRR957745,SRR957746,...,SRR5047099,SRR5047084,SRR5047171,SRR5047158,SRR5047135,SRR5047202,SRR5047105,SRR5047103,SRR5047142,SRR5047064
0,AAAAAAAAAAAAGAGCACAC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AAAAAAAAAAAGAGCACACA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAAAAAAAAAAGAGCACACC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AAAAAAAAAAAGAGCACACT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AAAAAAAAAAAGAGCACACG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
condition=str()
for each in list(HS_Oral.SRR_accession):
    condition=condition+ each +" != 0 and "
condition=condition[:-4]

In [121]:
#Create a dummie DF to try the commands before running them
dummie=HS_Oral_kmMatrix.limit(100).cache()
dummie.createOrReplaceTempView("dummie")
df2 = spark.sql("SELECT * FROM dummie WHERE "+condition).cache()


In [122]:
#This line is supposed to filter the rows of the human oral dataset where every entry is zero
HS_Oral_nozeroes = spark.sql("SELECT * FROM HS_Oral_kmMatrix WHERE "+condition).cache()

In [123]:
HS_Oral_nozeroes.createOrReplaceTempView("HS_Oral_nozeroes")

In [124]:
HS_Oral_kmMatrix.select(HS_Oral_kmMatrix['Kmer']).show(10)

+--------------------+
|                Kmer|
+--------------------+
|AAAAAAAAAAAAGAGCACAC|
|AAAAAAAAAAAGAGCACACA|
|AAAAAAAAAAAGAGCACACC|
|AAAAAAAAAAAGAGCACACT|
|AAAAAAAAAAAGAGCACACG|
|AAAAAAAAAAGAGCACACAA|
|AAAAAAAAAAGAGCACACAC|
|AAAAAAAAAAGAGCACACAT|
|AAAAAAAAAAGAGCACACAG|
|AAAAAAAAAAGAGCACACCA|
+--------------------+
only showing top 10 rows



## Prepare feature matrix for MLlib

In [96]:
aMetagenomes.unpersist();
HS_Oral_kmMatrix.unpersist();

In [ ]:
ML_input=

In [125]:
df = spark.createDataFrame([(Vectors.dense([6.0, 7.0, 0.0, 7.0, 6.0, 0.0]),),\
                            (Vectors.dense([0.0, 9.0, 6.0, 0.0, 5.0, 9.0]),),\
                            (Vectors.dense([0.0, 9.0, 3.0, 0.0, 5.0, 5.0]),),\
                            (Vectors.dense([0.0, 9.0, 8.0, 5.0, 6.0, 4.0]),),\
                            (Vectors.dense([8.0, 9.0, 6.0, 5.0, 4.0, 4.0]),),\
                            (Vectors.dense([8.0, 9.0, 6.0, 0.0, 0.0, 0.0]),)],\
                           ["features"])

In [129]:
df.toPandas()

,features
0,"[6.0, 7.0, 0.0, 7.0, 6.0, 0.0]"
1,"[0.0, 9.0, 6.0, 0.0, 5.0, 9.0]"
2,"[0.0, 9.0, 3.0, 0.0, 5.0, 5.0]"
3,"[0.0, 9.0, 8.0, 5.0, 6.0, 4.0]"
4,"[8.0, 9.0, 6.0, 5.0, 4.0, 4.0]"
5,"[8.0, 9.0, 6.0, 0.0, 0.0, 0.0]"


In [126]:
#Try variance threshold feature selection (k-mers with variance below a certain threshold are eliminated)


selector = VarianceThresholdSelector(varianceThreshold=8.0, outputCol="selectedFeatures")

result = selector.fit(df).transform(df)

print("Output: Features with variance lower than %f are removed." %
      selector.getVarianceThreshold())
result.show()


Output: Features with variance lower than 8.000000 are removed.
+--------------------+-----------------+
|            features| selectedFeatures|
+--------------------+-----------------+
|[6.0,7.0,0.0,7.0,...|[6.0,0.0,7.0,0.0]|
|[0.0,9.0,6.0,0.0,...|[0.0,6.0,0.0,9.0]|
|[0.0,9.0,3.0,0.0,...|[0.0,3.0,0.0,5.0]|
|[0.0,9.0,8.0,5.0,...|[0.0,8.0,5.0,4.0]|
|[8.0,9.0,6.0,5.0,...|[8.0,6.0,5.0,4.0]|
|[8.0,9.0,6.0,0.0,...|[8.0,6.0,0.0,0.0]|
+--------------------+-----------------+



## Export output as html

In [ ]:
import os

os.system('jupyter nbconvert --to html Preprocessing.ipynb')